In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe


class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(

            threshold = 0.9,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [4]:
def runAttack(model_wrapper, n, path, target_ds):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    svc = SVC(kernel='linear', probability=True)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn, 'SVC':svc}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn, svc

In [6]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


### BERT-based-uncased

In [7]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr, svc_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8105065666041276
RF 0.7326454033771107
KNN 0.6801125703564728
SVC 0.8030018761726079


In [9]:
# custom model wrapper for CLIP
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [10]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert_mr)

In [11]:
runAttack(bert_model_wrapper_lr_mr, -1, 'attack_results_new/bertmrattacklr09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklr09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5:   0%|                         | 5/1066 [01:06<3:55:19, 13.31s/it]textattack: Saving checkpoint under "checkpoints\1720073192179.ta.chkpt" at 2024-07-04 14:06:32 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 1 / 4 / 10:   1%|▏                      | 10/1066 [01:16<2:14:12,  7.63s/it]textattack: Saving checkpoint under "checkpoints\1720073201892.ta.chkpt" at 2024-07-04 14:06:41 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 3 / 4 / 15:   1%|▎                      | 15/1066 [01:35<1:52:04,  6.40s/it]textattack: Saving checkpoint under "checkpoints\1720073221611.ta.chkpt" at 2024-07-04 14:07:01 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 756 / 100 / 199 / 1055:  99%|██████████████▊| 1055/1066 [38:15<00:23,  2.18s/it]textattack: Saving checkpoint under "checkpoints\1720075420795.ta.chkpt" at 2024-07-04 14:43:40 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 756 / 100 / 200 / 1056:  99%|██████████████▊| 1056/1066 [38:15<00:21,  2.17s/it]

[Succeeded / Failed / Skipped / Total] 760 / 100 / 200 / 1060:  99%|██████████████▉| 1060/1066 [38:27<00:13,  2.18s/it]textattack: Saving checkpoint under "checkpoints\1720075433421.ta.chkpt" at 2024-07-04 14:43:53 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 761 / 102 / 202 / 1065: 100%|██████████████▉| 1065/1066 [38:39<00:02,  2.18s/it]textattack: Saving checkpoint under "checkpoints\1720075445625.ta.chkpt" at 2024-07-04 14:44:05 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 762 / 102 / 202 / 1066: 100%|███████████████| 1066/1066 [38:41<00:00,  2.18s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 762    |
| Number of failed attacks:     | 102    |
| Number of skipped attacks:    | 202    |
| Original accuracy:            | 81.05% |
| Accuracy under attack:        | 9.57%  |
| Attack success rate:          | 88.19% |
| Average perturbed word %:     | 17.17% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 60.95  |
+-------------------------------+--------+


In [12]:
runAttack(bert_model_wrapper_rf_mr, -1, 'attack_results_new/bertmrattackrf09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrf09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5:   0%|                         | 5/1066 [00:20<1:12:04,  4.08s/it]textattack: Saving checkpoint under "checkpoints\1720075467909.ta.chkpt" at 2024-07-04 14:44:27 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 1 / 3 / 7:   1%|▏                          | 7/1066 [00:20<51:51,  2.94s/it]

[Succeeded / Failed / Skipped / Total] 4 / 2 / 4 / 10:   1%|▏                        | 10/1066 [00:31<56:11,  3.19s/it]textattack: Saving checkpoint under "checkpoints\1720075479453.ta.chkpt" at 2024-07-04 14:44:39 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 3 / 5 / 15:   1%|▎                        | 15/1066 [00:48<56:34,  3.23s/it]textattack: Saving checkpoint under "checkpoints\1720075495983.ta.chkpt" at 2024-07-04 14:44:55 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 648 / 125 / 282 / 1055:  99%|██████████████▊| 1055/1066 [34:16<00:21,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1720077503733.ta.chkpt" at 2024-07-04 15:18:23 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 652 / 126 / 282 / 1060:  99%|██████████████▉| 1060/1066 [34:28<00:11,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1720077515830.ta.chkpt" at 2024-07-04 15:18:35 after 1060 attacks.
[Succeeded / Failed / Skipped / Total] 652 / 126 / 283 / 1061: 100%|██████████████▉| 1061/1066 [34:28<00:09,  1.95s/it]

[Succeeded / Failed / Skipped / Total] 654 / 126 / 285 / 1065: 100%|██████████████▉| 1065/1066 [34:31<00:01,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1720077519409.ta.chkpt" at 2024-07-04 15:18:39 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 655 / 126 / 285 / 1066: 100%|███████████████| 1066/1066 [34:33<00:00,  1.94s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 655    |
| Number of failed attacks:     | 126    |
| Number of skipped attacks:    | 285    |
| Original accuracy:            | 73.26% |
| Accuracy under attack:        | 11.82% |
| Attack success rate:          | 83.87% |
| Average perturbed word %:     | 14.85% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 59.59  |
+-------------------------------+--------+


In [13]:
runAttack(bert_model_wrapper_knn_mr, -1, 'attack_results_new/bertmrattackknn09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknn09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   0%|▏                          | 5/1066 [00:11<41:14,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1720077532445.ta.chkpt" at 2024-07-04 15:18:52 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:   1%|▏                          | 6/1066 [00:11<34:33,  1.96s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                        | 10/1066 [00:15<28:08,  1.60s/it]textattack: Saving checkpoint under "checkpoints\1720077536773.ta.chkpt" at 2024-07-04 15:18:56 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 2 / 6 / 15:   1%|▎                      | 15/1066 [00:54<1:04:01,  3.65s/it]textattack: Saving checkpoint under "checkpoints\1720077575603.ta.chkpt" at 2024-07-04 15:19:35 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 691 / 27 / 337 / 1055:  99%|███████████████▊| 1055/1066 [21:31<00:13,  1.22s/it]textattack: Saving checkpoint under "checkpoints\1720078811889.ta.chkpt" at 2024-07-04 15:40:11 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 694 / 28 / 338 / 1060:  99%|███████████████▉| 1060/1066 [21:42<00:07,  1.23s/it]textattack: Saving checkpoint under "checkpoints\1720078823745.ta.chkpt" at 2024-07-04 15:40:23 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 697 / 28 / 340 / 1065: 100%|███████████████▉| 1065/1066 [21:47<00:01,  1.23s/it]textattack: Saving checkpoint under "checkpoints\1720078827800.ta.chkpt" at 2024-07-04 15:40:27 after 1065 attacks.
[Succeeded / Failed / Skipped / Total] 697 / 28 / 341 / 1066: 100%|████████████████| 1066/1066 [21:47<00:00,  1.23s/it]

[Succeeded / Failed / Skipped / Total] 697 / 28 / 341 / 1066: 100%|████████████████| 1066/1066 [21:47<00:00,  1.23s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 697    |
| Number of failed attacks:     | 28     |
| Number of skipped attacks:    | 341    |
| Original accuracy:            | 68.01% |
| Accuracy under attack:        | 2.63%  |
| Attack success rate:          | 96.14% |
| Average perturbed word %:     | 9.27%  |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 40.27  |
+-------------------------------+--------+


In [14]:
runAttack(bert_model_wrapper_svc_mr, -1, 'attack_results_new/bertmrattacksvc09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvc09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5:   0%|                         | 5/1066 [00:18<1:05:44,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1720078846641.ta.chkpt" at 2024-07-04 15:40:46 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 1 / 3 / 7:   1%|▏                          | 7/1066 [00:18<47:11,  2.67s/it]

[Succeeded / Failed / Skipped / Total] 5 / 1 / 4 / 10:   1%|▏                        | 10/1066 [00:26<45:51,  2.61s/it]textattack: Saving checkpoint under "checkpoints\1720078854103.ta.chkpt" at 2024-07-04 15:40:54 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 3 / 4 / 15:   1%|▎                      | 15/1066 [00:55<1:05:04,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1720078883779.ta.chkpt" at 2024-07-04 15:41:23 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 752 / 96 / 207 / 1055:  99%|███████████████▊| 1055/1066 [39:23<00:24,  2.24s/it]textattack: Saving checkpoint under "checkpoints\1720081191492.ta.chkpt" at 2024-07-04 16:19:51 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 752 / 96 / 208 / 1056:  99%|███████████████▊| 1056/1066 [39:23<00:22,  2.24s/it]

[Succeeded / Failed / Skipped / Total] 755 / 97 / 208 / 1060:  99%|███████████████▉| 1060/1066 [39:37<00:13,  2.24s/it]textattack: Saving checkpoint under "checkpoints\1720081205162.ta.chkpt" at 2024-07-04 16:20:05 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 756 / 99 / 210 / 1065: 100%|███████████████▉| 1065/1066 [39:49<00:02,  2.24s/it]textattack: Saving checkpoint under "checkpoints\1720081217929.ta.chkpt" at 2024-07-04 16:20:17 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 757 / 99 / 210 / 1066: 100%|████████████████| 1066/1066 [39:51<00:00,  2.24s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 757    |
| Number of failed attacks:     | 99     |
| Number of skipped attacks:    | 210    |
| Original accuracy:            | 80.3%  |
| Accuracy under attack:        | 9.29%  |
| Attack success rate:          | 88.43% |
| Average perturbed word %:     | 17.29% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 60.95  |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [15]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr, svc_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.7945590994371482
RF 0.7682926829268293
KNN 0.7298311444652908
SVC 0.7851782363977486


In [17]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [18]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert_mr)

In [19]:
runAttack(disbert_model_wrapper_lr_mr, -1, 'attack_results_new/disbertmrattacklr09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklr09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5:   0%|▏                          | 5/1066 [00:16<56:54,  3.22s/it]textattack: Saving checkpoint under "checkpoints\1720081390881.ta.chkpt" at 2024-07-04 16:23:10 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 1 / 3 / 7:   1%|▏                          | 7/1066 [00:16<40:44,  2.31s/it]

[Succeeded / Failed / Skipped / Total] 6 / 1 / 3 / 10:   1%|▏                        | 10/1066 [00:22<40:11,  2.28s/it]textattack: Saving checkpoint under "checkpoints\1720081397627.ta.chkpt" at 2024-07-04 16:23:17 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 3 / 4 / 15:   1%|▎                        | 15/1066 [00:35<41:49,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1720081410607.ta.chkpt" at 2024-07-04 16:23:30 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 738 / 100 / 217 / 1055:  99%|██████████████▊| 1055/1066 [23:09<00:14,  1.32s/it]textattack: Saving checkpoint under "checkpoints\1720082764758.ta.chkpt" at 2024-07-04 16:46:04 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 738 / 100 / 218 / 1056:  99%|██████████████▊| 1056/1066 [23:10<00:13,  1.32s/it]

[Succeeded / Failed / Skipped / Total] 741 / 101 / 218 / 1060:  99%|██████████████▉| 1060/1066 [23:18<00:07,  1.32s/it]textattack: Saving checkpoint under "checkpoints\1720082773081.ta.chkpt" at 2024-07-04 16:46:13 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 743 / 103 / 219 / 1065: 100%|██████████████▉| 1065/1066 [23:29<00:01,  1.32s/it]textattack: Saving checkpoint under "checkpoints\1720082783992.ta.chkpt" at 2024-07-04 16:46:23 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 744 / 103 / 219 / 1066: 100%|███████████████| 1066/1066 [23:30<00:00,  1.32s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 744    |
| Number of failed attacks:     | 103    |
| Number of skipped attacks:    | 219    |
| Original accuracy:            | 79.46% |
| Accuracy under attack:        | 9.66%  |
| Attack success rate:          | 87.84% |
| Average perturbed word %:     | 17.62% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 59.31  |
+-------------------------------+--------+


In [20]:
runAttack(disbert_model_wrapper_rf_mr, -1, 'attack_results_new/disbertmrattackrf09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrf09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5:   0%|▏                          | 5/1066 [00:14<52:30,  2.97s/it]textattack: Saving checkpoint under "checkpoints\1720082799836.ta.chkpt" at 2024-07-04 16:46:39 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 1 / 2 / 6:   1%|▏                          | 6/1066 [00:14<43:53,  2.48s/it]

[Succeeded / Failed / Skipped / Total] 6 / 2 / 2 / 10:   1%|▏                        | 10/1066 [00:25<44:00,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1720082809997.ta.chkpt" at 2024-07-04 16:46:49 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 4 / 3 / 15:   1%|▎                        | 15/1066 [00:37<43:21,  2.48s/it]textattack: Saving checkpoint under "checkpoints\1720082822127.ta.chkpt" at 2024-07-04 16:47:02 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 665 / 144 / 246 / 1055:  99%|██████████████▊| 1055/1066 [25:29<00:15,  1.45s/it]textattack: Saving checkpoint under "checkpoints\1720084314126.ta.chkpt" at 2024-07-04 17:11:54 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 669 / 145 / 246 / 1060:  99%|██████████████▉| 1060/1066 [25:36<00:08,  1.45s/it]textattack: Saving checkpoint under "checkpoints\1720084321639.ta.chkpt" at 2024-07-04 17:12:01 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 672 / 146 / 247 / 1065: 100%|██████████████▉| 1065/1066 [25:47<00:01,  1.45s/it]textattack: Saving checkpoint under "checkpoints\1720084332786.ta.chkpt" at 2024-07-04 17:12:12 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 673 / 146 / 247 / 1066: 100%|███████████████| 1066/1066 [25:48<00:00,  1.45s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 673    |
| Number of failed attacks:     | 146    |
| Number of skipped attacks:    | 247    |
| Original accuracy:            | 76.83% |
| Accuracy under attack:        | 13.7%  |
| Attack success rate:          | 82.17% |
| Average perturbed word %:     | 17.36% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 66.78  |
+-------------------------------+--------+


In [21]:
runAttack(disbert_model_wrapper_knn_mr, -1, 'attack_results_new/disbertmrattackknn09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknn09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   0%|▏                          | 5/1066 [00:14<52:35,  2.97s/it]textattack: Saving checkpoint under "checkpoints\1720084348710.ta.chkpt" at 2024-07-04 17:12:28 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 1 / 2 / 7:   1%|▏                          | 7/1066 [00:14<37:46,  2.14s/it]

[Succeeded / Failed / Skipped / Total] 6 / 2 / 2 / 10:   1%|▏                        | 10/1066 [00:24<43:25,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1720084358514.ta.chkpt" at 2024-07-04 17:12:38 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 3 / 3 / 15:   1%|▎                        | 15/1066 [00:31<37:19,  2.13s/it]textattack: Saving checkpoint under "checkpoints\1720084365805.ta.chkpt" at 2024-07-04 17:12:45 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 624 / 146 / 285 / 1055:  99%|██████████████▊| 1055/1066 [23:36<00:14,  1.34s/it]textattack: Saving checkpoint under "checkpoints\1720085750175.ta.chkpt" at 2024-07-04 17:35:50 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 626 / 148 / 286 / 1060:  99%|██████████████▉| 1060/1066 [23:44<00:08,  1.34s/it]textattack: Saving checkpoint under "checkpoints\1720085758202.ta.chkpt" at 2024-07-04 17:35:58 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 629 / 148 / 288 / 1065: 100%|██████████████▉| 1065/1066 [23:49<00:01,  1.34s/it]textattack: Saving checkpoint under "checkpoints\1720085763467.ta.chkpt" at 2024-07-04 17:36:03 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 630 / 148 / 288 / 1066: 100%|███████████████| 1066/1066 [23:50<00:00,  1.34s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 630    |
| Number of failed attacks:     | 148    |
| Number of skipped attacks:    | 288    |
| Original accuracy:            | 72.98% |
| Accuracy under attack:        | 13.88% |
| Attack success rate:          | 80.98% |
| Average perturbed word %:     | 11.56% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 60.49  |
+-------------------------------+--------+


In [22]:
runAttack(disbert_model_wrapper_svc_mr, -1, 'attack_results_new/disbertmrattacksvc09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvc09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 2 / 2 / 1 / 5:   0%|▏                          | 5/1066 [00:16<56:43,  3.21s/it]textattack: Saving checkpoint under "checkpoints\1720085779950.ta.chkpt" at 2024-07-04 17:36:19 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 2 / 3 / 7:   1%|▏                          | 7/1066 [00:16<40:38,  2.30s/it]

[Succeeded / Failed / Skipped / Total] 5 / 2 / 3 / 10:   1%|▏                        | 10/1066 [00:23<42:03,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1720085787806.ta.chkpt" at 2024-07-04 17:36:27 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 4 / 4 / 15:   1%|▎                        | 15/1066 [00:36<42:43,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1720085800497.ta.chkpt" at 2024-07-04 17:36:40 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 729 / 100 / 226 / 1055:  99%|██████████████▊| 1055/1066 [23:58<00:15,  1.36s/it]textattack: Saving checkpoint under "checkpoints\1720087202723.ta.chkpt" at 2024-07-04 18:00:02 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 729 / 100 / 227 / 1056:  99%|██████████████▊| 1056/1066 [23:58<00:13,  1.36s/it]

[Succeeded / Failed / Skipped / Total] 733 / 100 / 227 / 1060:  99%|██████████████▉| 1060/1066 [24:05<00:08,  1.36s/it]textattack: Saving checkpoint under "checkpoints\1720087209842.ta.chkpt" at 2024-07-04 18:00:09 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 736 / 101 / 228 / 1065: 100%|██████████████▉| 1065/1066 [24:18<00:01,  1.37s/it]textattack: Saving checkpoint under "checkpoints\1720087222622.ta.chkpt" at 2024-07-04 18:00:22 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 737 / 101 / 228 / 1066: 100%|███████████████| 1066/1066 [24:19<00:00,  1.37s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 737    |
| Number of failed attacks:     | 101    |
| Number of skipped attacks:    | 228    |
| Original accuracy:            | 78.61% |
| Accuracy under attack:        | 9.47%  |
| Attack success rate:          | 87.95% |
| Average perturbed word %:     | 17.74% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 59.7   |
+-------------------------------+--------+


### CLIP

In [23]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [24]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr, svc_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg 0.7495309568480301
RF 0.7054409005628518
KNN 0.6885553470919324
SVC 0.7551594746716698


In [25]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [26]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip_mr)

In [27]:
runAttack(clip_model_wrapper_lr_mr, -1, 'attack_results_new/clipmrattacklr09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklr09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:17<1:01:40,  3.49s/it]textattack: Saving checkpoint under "checkpoints\1720087679631.ta.chkpt" at 2024-07-04 18:07:59 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▏                          | 6/1066 [00:17<51:28,  2.91s/it]

[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:   1%|▏                        | 10/1066 [00:24<43:45,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1720087687050.ta.chkpt" at 2024-07-04 18:08:07 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 1 / 1 / 15:   1%|▎                       | 15/1066 [00:36<43:07,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1720087699114.ta.chkpt" at 2024-07-04 18:08:19 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 731 / 59 / 265 / 1055:  99%|███████████████▊| 1055/1066 [22:25<00:14,  1.28s/it]textattack: Saving checkpoint under "checkpoints\1720089007666.ta.chkpt" at 2024-07-04 18:30:07 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 731 / 59 / 266 / 1056:  99%|███████████████▊| 1056/1066 [22:25<00:12,  1.27s/it]

[Succeeded / Failed / Skipped / Total] 735 / 59 / 266 / 1060:  99%|███████████████▉| 1060/1066 [22:32<00:07,  1.28s/it]textattack: Saving checkpoint under "checkpoints\1720089014571.ta.chkpt" at 2024-07-04 18:30:14 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 739 / 59 / 267 / 1065: 100%|███████████████▉| 1065/1066 [22:44<00:01,  1.28s/it]textattack: Saving checkpoint under "checkpoints\1720089026932.ta.chkpt" at 2024-07-04 18:30:26 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 740 / 59 / 267 / 1066: 100%|████████████████| 1066/1066 [22:45<00:00,  1.28s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 740    |
| Number of failed attacks:     | 59     |
| Number of skipped attacks:    | 267    |
| Original accuracy:            | 74.95% |
| Accuracy under attack:        | 5.53%  |
| Attack success rate:          | 92.62% |
| Average perturbed word %:     | 15.89% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 54.45  |
+-------------------------------+--------+


In [28]:
runAttack(clip_model_wrapper_rf_mr, -1, 'attack_results_new/clipmrattackrf09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrf09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 2 / 0 / 5:   0%|                         | 5/1066 [00:33<1:58:36,  6.71s/it]textattack: Saving checkpoint under "checkpoints\1720089061407.ta.chkpt" at 2024-07-04 18:31:01 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 2 / 2 / 7:   1%|▏                        | 7/1066 [00:33<1:24:54,  4.81s/it]

[Succeeded / Failed / Skipped / Total] 6 / 2 / 2 / 10:   1%|▏                      | 10/1066 [00:43<1:16:18,  4.34s/it]textattack: Saving checkpoint under "checkpoints\1720089071228.ta.chkpt" at 2024-07-04 18:31:11 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 2 / 3 / 15:   1%|▎                     | 15/1066 [01:00<1:11:03,  4.06s/it]textattack: Saving checkpoint under "checkpoints\1720089088728.ta.chkpt" at 2024-07-04 18:31:28 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 615 / 128 / 312 / 1055:  99%|██████████████▊| 1055/1066 [37:21<00:23,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1720091269734.ta.chkpt" at 2024-07-04 19:07:49 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 619 / 128 / 313 / 1060:  99%|██████████████▉| 1060/1066 [37:28<00:12,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1720091276119.ta.chkpt" at 2024-07-04 19:07:56 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 623 / 128 / 314 / 1065: 100%|██████████████▉| 1065/1066 [37:38<00:02,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1720091286454.ta.chkpt" at 2024-07-04 19:08:06 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 624 / 128 / 314 / 1066: 100%|███████████████| 1066/1066 [37:39<00:00,  2.12s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 624    |
| Number of failed attacks:     | 128    |
| Number of skipped attacks:    | 314    |
| Original accuracy:            | 70.54% |
| Accuracy under attack:        | 12.01% |
| Attack success rate:          | 82.98% |
| Average perturbed word %:     | 14.62% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 61.24  |
+-------------------------------+--------+


In [29]:
runAttack(clip_model_wrapper_knn_mr, -1, 'attack_results_new/clipmrattackknn09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknn09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   0%|                         | 5/1066 [00:53<3:07:43, 10.62s/it]textattack: Saving checkpoint under "checkpoints\1720091340681.ta.chkpt" at 2024-07-04 19:09:00 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 5 / 2 / 10:   1%|▏                      | 10/1066 [01:09<2:02:58,  6.99s/it]textattack: Saving checkpoint under "checkpoints\1720091357474.ta.chkpt" at 2024-07-04 19:09:17 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 7 / 3 / 15:   1%|▎                      | 15/1066 [01:32<1:47:28,  6.14s/it]textattack: Saving checkpoint under "checkpoints\1720091379643.ta.chkpt" at 2024-07-04 19:09:39 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 572 / 152 / 331 / 1055:  99%|██████████████▊| 1055/1066 [36:20<00:22,  2.07s/it]textattack: Saving checkpoint under "checkpoints\1720093468277.ta.chkpt" at 2024-07-04 19:44:28 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 576 / 153 / 331 / 1060:  99%|██████████████▉| 1060/1066 [36:30<00:12,  2.07s/it]textattack: Saving checkpoint under "checkpoints\1720093477857.ta.chkpt" at 2024-07-04 19:44:37 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 579 / 154 / 332 / 1065: 100%|██████████████▉| 1065/1066 [36:38<00:02,  2.06s/it]textattack: Saving checkpoint under "checkpoints\1720093486558.ta.chkpt" at 2024-07-04 19:44:46 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 580 / 154 / 332 / 1066: 100%|███████████████| 1066/1066 [36:39<00:00,  2.06s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 580    |
| Number of failed attacks:     | 154    |
| Number of skipped attacks:    | 332    |
| Original accuracy:            | 68.86% |
| Accuracy under attack:        | 14.45% |
| Attack success rate:          | 79.02% |
| Average perturbed word %:     | 10.8%  |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 60.39  |
+-------------------------------+--------+


In [30]:
runAttack(clip_model_wrapper_svc_mr, -1, 'attack_results_new/clipmrattacksvc09.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvc09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:27<1:37:06,  5.49s/it]textattack: Saving checkpoint under "checkpoints\1720093514674.ta.chkpt" at 2024-07-04 19:45:14 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▏                        | 6/1066 [00:27<1:21:10,  4.59s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   1%|▏                      | 10/1066 [00:37<1:05:58,  3.75s/it]textattack: Saving checkpoint under "checkpoints\1720093524707.ta.chkpt" at 2024-07-04 19:45:24 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   1%|▎                     | 15/1066 [00:52<1:01:43,  3.52s/it]textattack: Saving checkpoint under "checkpoints\1720093540074.ta.chkpt" at 2024-07-04 19:45:40 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 728 / 66 / 261 / 1055:  99%|███████████████▊| 1055/1066 [33:18<00:20,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1720095485996.ta.chkpt" at 2024-07-04 20:18:05 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 728 / 66 / 262 / 1056:  99%|███████████████▊| 1056/1066 [33:18<00:18,  1.89s/it]

[Succeeded / Failed / Skipped / Total] 732 / 66 / 262 / 1060:  99%|███████████████▉| 1060/1066 [33:26<00:11,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1720095493988.ta.chkpt" at 2024-07-04 20:18:13 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 736 / 66 / 263 / 1065: 100%|███████████████▉| 1065/1066 [33:40<00:01,  1.90s/it]textattack: Saving checkpoint under "checkpoints\1720095507851.ta.chkpt" at 2024-07-04 20:18:27 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 737 / 66 / 263 / 1066: 100%|████████████████| 1066/1066 [33:41<00:00,  1.90s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 737    |
| Number of failed attacks:     | 66     |
| Number of skipped attacks:    | 263    |
| Original accuracy:            | 75.33% |
| Accuracy under attack:        | 6.19%  |
| Attack success rate:          | 91.78% |
| Average perturbed word %:     | 15.49% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 54.43  |
+-------------------------------+--------+
